## Splitting Evaluation
The objective of this notebook is to obtain a standard way of evaluating models. For each person, the model is expected to predict a probability.  The referal is then evaluated under what time period it occurs. 




In [0]:
%reload_ext autoreload
%autoreload 2
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn


In [0]:
import sys
import pandas as pd
import datetime
from pathlib import Path
import numpy as np
pd.set_option('display.max_columns', 9999)
from sklearn.metrics import precision_score, accuracy_score, recall_score, balanced_accuracy_score, f1_score
from sklearn.metrics import roc_auc_score, log_loss
from sklearn.metrics import roc_curve

In [0]:
#This could be used for benchmarking data
#100 People for 5.5 years. 
path=Path('../data')
file='test.csv' # has just 10 people for 2 years
#file='long-5-1000-10-5-2-True-10-5-2-True-2-101-5-0-0.csv'
df=pd.read_csv('https://raw.githubusercontent.com/HealthINCITE/patient_panel/master/data/test.csv')
df

,id,yrm,cad0,cad1,dv9,target
0,1000,201601,1,1,0,0
1,1000,201602,1,1,0,0
2,1000,201603,1,1,0,0
3,1000,201604,1,1,0,0
4,1000,201605,1,1,0,0
...,...,...,...,...,...,...
235,1009,201708,0,1,1,0
236,1009,201709,0,1,1,0
237,1009,201710,0,1,1,0
238,1009,201711,0,1,1,0


## Train test split based on time window.


In [0]:
#Define the function
def train_test_split(df, date_col, date_format, split_time):
    split =pd.Timestamp(split_time)
    #Let's convert this to datetime while we are at it. 
    df['yrm'] = pd.to_datetime(df[date_col], format=date_format)
    train=df.loc[df['yrm']<split]
    test=df.loc[df['yrm']>split]
    return train, test

#Run the Fuction
train, test = train_test_split(df, 'yrm', '%Y%m', datetime.date(2016, 12,30)) 
test

,id,yrm,cad0,cad1,dv9,target
12,1000,2017-01-01,1,1,0,0
13,1000,2017-02-01,1,1,0,0
14,1000,2017-03-01,1,1,0,0
15,1000,2017-04-01,1,1,0,0
16,1000,2017-05-01,1,1,0,0
...,...,...,...,...,...,...
235,1009,2017-08-01,0,1,1,0
236,1009,2017-09-01,0,1,1,0
237,1009,2017-10-01,0,1,1,0
238,1009,2017-11-01,0,1,1,0


### Predictions 
The predictions are easy to assess for the toy model. 
For the toy model:
    - The first 4 individuals are not referrals.
    - The next 2 are positive in the first three months.
    - The next 2 are positive in the first 6 months.
    - The final 2 are positive in the 12th month. 

We set the windows according to the following. 
`windows= [[0,3], [0,6], [0,12]]`

In [0]:
probabilities=np.array([0,.05,.1,0,.9,.9,.8,.8,.9,.9])
threshold=0.5
predictions = np.where(probabilities > 0.5, 1, 0)
predictions

array([0, 0, 0, 0, 1, 1, 1, 1, 1, 1])

In [0]:
def score_windows(test, probabilities, threshold, windows):
    """
    test = test dataframe
    
    """
    ir=test.pivot_table(index='id', columns='yrm', values='target', aggfunc='sum')
    c=ir.columns
    df=pd.DataFrame() #final results
    predictions=np.where(probabilities > 0.5, 1, 0) 
    row=0
    # Loop through the windows
    for w in windows:
        sl=slice(w[0],w[1])
        y= ir.iloc[:,sl].sum(axis=1)
        label=c[w[0]].strftime('%Y%m')+'-'+c[w[1]-1].strftime('%Y%m')
        df.loc[row, 'range']=label
        df.loc[row, 'log_loss'] = log_loss(y, probabilities)
        df.loc[row, 'precision']=precision_score(y, predictions)
        df.loc[row, 'recall']=recall_score(y, predictions)
        df.loc[row, 'accuracy']=accuracy_score(y, predictions)
        df.loc[row, 'balanced_accuracy']=balanced_accuracy_score(y, predictions)
        df.loc[row, 'f1']=f1_score(y, predictions)
        row=row+1
    return df

#define the windows.  For example [0,3] is including between 0-3 months.
windows= [[0,3], [0,6], [0,12]]
#Score windows
score_windows(test,probabilities,threshold,windows)


,range,log_loss,precision,recall,accuracy,balanced_accuracy,f1
0,201701-201703,0.819142,0.333333,1.0,0.6,0.750000,0.5
1,201701-201706,0.541883,0.666667,1.0,0.8,0.833333,0.8
2,201701-201712,0.102438,1.000000,1.0,1.0,1.000000,1.0


## Null Model 
The null model here is just that there are referrals. 

In [0]:
null=np.array([0,0,0,0,0,0,0,0,0,0])
score_windows(test,null,threshold,windows)


,range,log_loss,precision,recall,accuracy,balanced_accuracy,f1
0,201701-201703,6.907755,0.0,0.0,0.8,0.5,0.0
1,201701-201706,13.815511,0.0,0.0,0.6,0.5,0.0
2,201701-201712,20.723266,0.0,0.0,0.4,0.5,0.0
